#### NOTA: En este cuaderno se crearan las Tablas `compra_header` y `compra_detail` del Datawarehouse

***
### Importando Librerias

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import count, when, isnull


***
Creando Session de Spark

In [3]:
spark = SparkSession.builder.appName('compra_header-compra_detail').getOrCreate()

***
Importando Conexion a Contenedor `mod4_mysql`

In [4]:
from db.connection import connec_db

***
Trabajando el csv asociado a  `compra`

In [4]:
compra = spark.read.csv("Data/Compra.csv", sep=",", header=True, inferSchema=True)

***
Visualizando como vienen los Datos

In [5]:
compra.show(5)

+--------+----------+----------+--------+------+-----------+
|IdCompra|     Fecha|IdProducto|Cantidad|Precio|IdProveedor|
+--------+----------+----------+--------+------+-----------+
|       1|2015-01-30|     42832|      13|560.51|         12|
|       2|2015-01-30|     42833|      11|497.58|          7|
|       3|2015-01-30|     42834|       1| 588.5|          6|
|       4|2015-01-30|     42835|       9|567.66|         14|
|       5|2015-01-30|     42839|      14|231.31|          2|
+--------+----------+----------+--------+------+-----------+
only showing top 5 rows



***
Visualizando la Inferencia del Schema

In [6]:
compra.printSchema()

root
 |-- IdCompra: integer (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- IdProducto: integer (nullable = true)
 |-- Cantidad: integer (nullable = true)
 |-- Precio: double (nullable = true)
 |-- IdProveedor: integer (nullable = true)



***
Buscando Valores Faltantes

In [7]:
# Contar valores nulos por columna
compra.agg(*[count(when(isnull(c), c)).alias(c) for c in compra.columns]).show()



+--------+-----+----------+--------+------+-----------+
|IdCompra|Fecha|IdProducto|Cantidad|Precio|IdProveedor|
+--------+-----+----------+--------+------+-----------+
|       0|    0|         0|       0|     0|          0|
+--------+-----+----------+--------+------+-----------+



Creo los dataframes `compra_header` y `compra_detail` a partir de `compra`

In [8]:
compra_header = compra.select("IdCompra", "Fecha", "IdProveedor")

In [9]:
compra_detail = compra.select("IdCompra", "IdProducto", "Cantidad", "Precio")

Visualizo como se ven los nuevos Dataframes

In [10]:
compra_header.show()

+--------+----------+-----------+
|IdCompra|     Fecha|IdProveedor|
+--------+----------+-----------+
|       1|2015-01-30|         12|
|       2|2015-01-30|          7|
|       3|2015-01-30|          6|
|       4|2015-01-30|         14|
|       5|2015-01-30|          2|
|       6|2015-01-30|         13|
|       7|2015-01-30|          4|
|       8|2015-01-30|          4|
|       9|2015-01-30|         12|
|      10|2015-01-30|          6|
|      11|2015-01-30|          3|
|      12|2015-01-30|          7|
|      13|2015-01-30|         11|
|      14|2015-01-30|          6|
|      15|2015-01-30|          2|
|      16|2015-01-30|          5|
|      17|2015-01-30|         13|
|      18|2015-01-30|          7|
|      19|2015-01-30|          8|
|      20|2015-01-30|         13|
+--------+----------+-----------+
only showing top 20 rows



In [11]:
compra_detail.show()

+--------+----------+--------+-------+
|IdCompra|IdProducto|Cantidad| Precio|
+--------+----------+--------+-------+
|       1|     42832|      13| 560.51|
|       2|     42833|      11| 497.58|
|       3|     42834|       1|  588.5|
|       4|     42835|       9| 567.66|
|       5|     42839|      14| 231.31|
|       6|     42840|      14| 232.07|
|       7|     42841|       8| 236.98|
|       8|     42842|       4| 255.33|
|       9|     42845|       5| 578.61|
|      10|     42855|       1| 809.04|
|      11|     42857|       8| 379.77|
|      12|     42858|      11| 365.37|
|      13|     42860|       5| 447.15|
|      14|     42862|       4| 310.02|
|      15|     42984|       7| 173.64|
|      16|     42907|       6| 316.37|
|      17|     42908|       4|1081.16|
|      18|     42911|       8|1409.06|
|      19|     42914|       4|  982.0|
|      20|     42915|      17|1225.11|
+--------+----------+--------+-------+
only showing top 20 rows



In [12]:
del compra

***
Creando la Conexion

In [9]:
engine = connec_db()

***
Pasando los Dataframes a Dataframes de Pandas  y vaciando en sus respectivas Tablas

In [13]:
# IMPORTANDO LIBRERIA PANDAS
import pandas as pd

***
Vaciando el Dataframe a la Tabla `compra_header`

In [14]:
pandas_df_compra_h = compra_header.toPandas()


In [12]:
pandas_df_compra_h.to_sql(name="compra_header", con=engine, index=False, if_exists="append")

11539

In [15]:
pandas_df_compra_h.to_csv("csv_practica/compra_header.csv", index=False)

In [16]:
del pandas_df_compra_h
del compra_header

***
Vaciando el Dataframe a la Tabla `compra_detail`

In [17]:
pandas_df_compra_d = compra_detail.toPandas()

In [15]:
pandas_df_compra_d.to_sql(name="compra_detail", con=engine, index=False, if_exists="append")

11539

In [18]:
pandas_df_compra_d.to_csv("csv_practica/compra_detail.csv", index=False)

In [19]:
del compra_detail
del pandas_df_compra_d

In [20]:
spark.stop()